<center>
<img src="https://raw.githubusercontent.com/FUlyankin/r_probability/master/end_seminars/sem_5/pEbNtDxzx0Q.jpg" height="500" width="500"> 


# R для тервера и матстата

## 4.2 Доверительные интервалы


Данный ноутбук является домашкой по курсу «R для теории вероятностей и математической статистики» (РАНХиГС, 2019). Автор ноутбука [вот этот парень по имени Филипп.](https://vk.com/ppilif) Если у вас для него есть деньги, слава или женщины, он от этого всего не откажется. Ноутбук распространяется на условиях лицензии [Creative Commons Attribution-Share Alike 4.0.](https://creativecommons.org/licenses/by-sa/4.0/) При использовании обязательно упоминание автора курса и аффилиации. При наличии технической возможности необходимо также указать активную гиперссылку на [страницу курса.](https://fulyankin.github.io/R_probability/) На ней можно найти другие материалы. Фрагменты кода, включенные в этот notebook, публикуются как [общественное достояние.](https://creativecommons.org/publicdomain/zero/1.0/)

------------------------

В этой тетрадке мы обсудим почему среднее это очень круто. Мы сделаем переход от точечных оценок к интервальным и попытаемся разобраться как строятся доверительные интервалы, а также обсудим __дельта-метод.__ Итак, план: 

1. Мощь средних, асимптотические доверительные интервалы 
2. Дельта-метод 
3. Всё и сразу в задачке про Свету
4. Точные доверительные интервалы


# 1. Ещё раз про схему матстата 

# 2. Мощь средних и асимптотические доверительные интервалы

# 3. Асимптотические доверительные интервалы для долей

# 4. Дельта-метод и Ульяна

# 5. Кто наши союзники? 

# 6. Точные доверительные интервалы 

# 7.  Недвижимая недвижимость! 


В [табличке](https://yadi.sk/i/hDxMJ0fw3VSDmu) лежит информация о стоимости квартир в Москве и о основных параметрах этих квартир.

__Описание переменных:__

```
n – номер квартиры по порядку
price – цена квартиры в $1000
totsp – общая площадь квартиры, кв.м.
livesp жилая площадь квартиры, кв.м.
kitsp – площадь кухни, кв.м.
dist – расстояние от центра в км.
metrdist – расстояние до метро в минутах
walk – 1 – пешком от метро, 0 – на транспорте
brick 1 – кирпичный, монолит ж/б, 0 – другой
floor 1 – этаж кроме первого и последнего, 0 – иначе.
code – число от 1 до 8, при помощи которого мы группируем наблюдения по
подвыборкам:
1. Наблюдения сгруппированы на севере, вокруг Калужско-Рижской линии
метрополитена
2. Север, вокруг Серпуховско-Тимирязевской линии метрополитена
3. Северо-запад, вокруг Замоскворецкой линии метрополитена
4. Северо-запад, вокруг Таганско-Краснопресненской линии метрополитена
5. Юго-восток, вокруг Люблинской линии метрополитена
6. Юго-восток, вокруг Таганско-Краснопресненской линии метрополитена
7. Восток, вокруг Калиниской линии метрополитена
8. Восток, вокруг Арбатско-Покровской линии метрополитена
```

__[a]__ Построить асимптотический доверительный интервал для средней стоимости квартиры. Построить точный доверительный интервал для средней стоимости квартиры. Какой из них оказался короче? Почему? 

__[b]__ Построить асимптотические доверительные интервалы для всех районов. В какие из них попало общее среднее? Постройте красивую визулизацию.

__[c]__ Постройте по всем районам асимптотические доверительные интервалы для разницы в средней стоимости однокомнатных и двукомнатных квартир. Где эта разница значима? На каком уровне значимости? 

__[d]__ 

http://rpubs.com/rodsveiga/252173